<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.20.1
Pandas:  1.2.4


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [4]:
# wrong:
bikes = pd.read_csv('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_csv('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_csv('bikeshare.csv', header = 0)
print(bikes.head())

        0           1       2   3     4   5        6        7           8   \
0  instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday   
1        1  2011-01-01       1   0     1   0        0        6           0   
2        2  2011-01-01       1   0     1   1        0        6           0   
3        3  2011-01-01       1   0     1   2        0        6           0   
4        4  2011-01-01       1   0     1   3        0        6           0   

           9     10      11    12         13      14          15   16  
0  weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
1           1  0.24  0.2879  0.81          0       3          13   16  
2           1  0.22  0.2727   0.8          0       8          32   40  
3           1  0.22  0.2727   0.8          0       5          27   32  
4           1  0.24  0.2879  0.75          0       3          10   13  

   1  2011-01-01  1.1  0  1.2  0.1  0.2  6  0.3  1.3  0.24  0.2879  0.81  \
0  2  2011-01-01    1 

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [6]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv', header = None)
df_bikes=pd.DataFrame(bikes)

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [7]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [11]:
df = pd.read_excel('Iris.xls' )
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [14]:
#ANSWER
df.dropna()
df.isnull().sum()
df = pd.read_excel('Iris.xls', sheet_name = 'Data', header = 0, usecols = [0, 1, 2, 3, 4])
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [15]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-12-08T05:18:04.030Z,18.933332,-155.187164,10.04,2.84,ml,37.0,250,NaN,0.17,...,2021-12-08T05:23:34.190Z,"42 km SE of Pāhala, Hawaii",earthquake,0.89,0.76,0.350,13,automatic,hv,hv
1,2021-12-08T05:15:24.974Z,44.367400,-129.379400,10.00,5.50,mww,NaN,169,1.474,0.71,...,2021-12-08T05:48:36.516Z,off the coast of Oregon,earthquake,8.40,1.90,0.030,109,reviewed,us,us
2,2021-12-08T04:54:16.108Z,44.522800,-129.407500,10.00,4.30,mb,NaN,212,4.012,0.82,...,2021-12-08T05:11:40.040Z,off the coast of Oregon,earthquake,10.80,2.00,0.122,19,reviewed,us,us
3,2021-12-08T04:53:12.659Z,44.387700,-129.497300,10.00,4.00,mb,NaN,212,1.442,0.88,...,2021-12-08T05:26:17.040Z,off the coast of Oregon,earthquake,8.30,2.00,0.172,11,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [16]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

[                         Bank NameBank           CityCity StateSt  CertCert  \
 0                    Almena State Bank             Almena      KS     15426   
 1           First City Bank of Florida  Fort Walton Beach      FL     16748   
 2                 The First State Bank      Barboursville      WV     14361   
 3                   Ericson State Bank            Ericson      NE     18265   
 4     City National Bank of New Jersey             Newark      NJ     21111   
 ..                                 ...                ...     ...       ...   
 558                 Superior Bank, FSB           Hinsdale      IL     32646   
 559                Malta National Bank              Malta      OH      6629   
 560    First Alliance Bank & Trust Co.         Manchester      NH     34264   
 561  National State Bank of Metropolis         Metropolis      IL      3815   
 562                   Bank of Honolulu           Honolulu      HI     21029   
 
                  Acquiring Institutio

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [17]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [31]:
#ANSWER
#ANSWER

bikes.columns = bikes.iloc[0]
bikes = bikes.iloc[1: , :]
bikes
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1


What are the column names in `bikes`?

In [23]:
#ANSWER
#ANSWER
bikes.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], dtype='int64')

What are the data types of these columns?

In [20]:
#ANSWER
#ANSWER
bikes.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
dtype: object

What is the (row) index for this DataFrame?

In [21]:
#ANSWER
#ANSWER
bikes.index

RangeIndex(start=0, stop=17380, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [32]:
#ANSWER
#ANSWER
print(bikes['atemp'].head())
print(type(bikes['atemp']))
print(bikes[['atemp']].head())
print(type(bikes[['atemp']]))

1    0.2879
2    0.2727
3    0.2727
4    0.2879
5    0.2879
Name: atemp, dtype: object
<class 'pandas.core.series.Series'>
0   atemp
1  0.2879
2  0.2727
3  0.2727
4  0.2879
5  0.2879
<class 'pandas.core.frame.DataFrame'>


How would we use object notation to show the first 4 rows of `atemp`?

In [33]:
#ANSWER
print(bikes.atemp[:4])


1    0.2879
2    0.2727
3    0.2727
4    0.2879
Name: atemp, dtype: object


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [35]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: 0

What is the correct way to access the 1st row of the DataFrame by its index?

In [36]:
#ANSWER
bikes[:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [41]:
#ANSWER
bikes.iloc[:, 1]

1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
5        2011-01-01
            ...    
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
17379    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [42]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [45]:
#ANSWER
bikes['windspeed'].isnull().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [44]:
#ANSWER

bikes.isnull().sum()

0
instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [46]:
#ANSWER
def divide(dividend, divisor):
    if divisor == 0:
        quotient = np.nan
    else:
        quotient = dividend / divisor
    return (quotient)

print(divide(1, 0))


nan


Apply the Pandas `isna` function to the following data objects:

In [47]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [48]:
#ANSWER
#ANSWER
print(pd.isna(x), pd.isna(y))

False True


In [49]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [50]:
#ANSWER
#ANSWER
print(pd.isna(array))


[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [51]:
#ANSWER
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [52]:
#ANSWER
print('instant:', bikes['instant'].min(), 'to', bikes['instant'].max())
print('dteday:', bikes['dteday'].min(), 'to', bikes['dteday'].max())
print('windspeed:', bikes['windspeed'].min(), 'to', bikes['windspeed'].max())

instant: 1 to 9999
dteday: 2011-01-01 to 2012-12-31
windspeed: 0 to 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [0]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [64]:
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [65]:
#ANSWER
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [66]:
type(bikes['atemp'])

pandas.core.series.Series

In [67]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
13613,13613,2012-07-26,3,1,7,9,0,4,1,2,0.72,0.6818,0.7,0.1343,55,289,344
10647,10647,2012-03-24,2,1,3,17,0,6,0,3,0.5,0.4848,0.88,0.2239,47,68,115
1298,1298,2011-02-27,1,0,2,6,0,0,0,1,0.24,0.2879,0.87,0,2,1,3
9416,9416,2012-02-02,1,1,2,5,0,4,1,3,0.4,0.4091,0.82,0,0,19,19
3572,3572,2011-06-02,2,0,6,17,0,4,1,1,0.76,0.6515,0.2,0.5224,88,484,572


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [71]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

TypeError: can only concatenate str (not "float") to str

What is `atemp_level`?

In [70]:
#ANSWER
print(type(atemp_level))

NameError: name 'atemp_level' is not defined

Here is a random sample of `atemp_level`:

In [0]:
atemp_level.sample(5)          

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [0]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [0]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes[(bikes.atemp_level == 'mild') & (bikes.season == 2)].season.count()  # Nb. could have used any column before count() in this case


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [0]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [0]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

## Check Column Names

Check column names and clean.

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



